In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights

from cnn_utils import (
    SealDataset, 
    display_result_metrics,
    generate_predictions_pytorch, 
    get_labels, 
)

In [ ]:
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using: ", device)
torch.cuda.empty_cache()

In [ ]:
model_name ="ImageClassifierPytorch9"
path = f"../Models/PyTorch/{model_name}"

# Load Model with pre-trained weights
efficientnet = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)

# Add final layer to predict 2 classes
efficientnet.classifier[1] = nn.Linear(in_features=1280, out_features=2)

# Load trained weights
efficientnet.load_state_dict(torch.load(path))

# Put on device
efficientnet = efficientnet.to(device)

In [ ]:
# Load Data
data_path = "../Data"

train_img_data = np.load(f"{data_path}/train_images.npy", allow_pickle=True)
train_bb_data = np.load(f"{data_path}/train_bb_data.npy", allow_pickle=True)

val_img_data  = np.load(f"{data_path}/val_images.npy", allow_pickle=True)
val_bb_data = np.load(f"{data_path}/val_bb_data.npy", allow_pickle=True)

test_img_data  = np.load(f"{data_path}/test_images.npy", allow_pickle=True)
test_bb_data = np.load(f"{data_path}/test_bb_data.npy", allow_pickle=True)

In [ ]:
# Get labels for sub-images
seal_threshold = .3

training_labels = get_labels(train_bb_data, seal_threshold)
val_labels = get_labels(val_bb_data, seal_threshold)
test_labels = get_labels(test_bb_data, seal_threshold)

In [ ]:
# Convert to Tensor
train_labels = torch.tensor(training_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Create Dataset object
train_data = SealDataset(train_img_data, training_labels)
valid_data = SealDataset(val_img_data, val_labels)
test_data = SealDataset(test_img_data, test_labels)

# Create Data Loader
train_loader = DataLoader(dataset = train_data, batch_size=10, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=50, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=50, shuffle=True)

In [ ]:
def evaluate_model(model, loader:DataLoader, device:str) -> None:
    """Get predicted labels from model and displays evaluation metrics

    Args:
        model (torchvision model): Model to generate predictions with
        loader (DataLoader): DataLoader object containing data to evaluate
        device (str): Can either be 'cpu' or 'cuda', refers to CPU or GPU usage respectively
    """
    actual_labels, predicted_labels = generate_predictions_pytorch(model, loader, device)
    display_result_metrics(actual_labels, predicted_labels)

In [ ]:
evaluate_model(efficientnet, train_loader, device)

In [ ]:
evaluate_model(efficientnet, valid_loader, device)

In [ ]:
evaluate_model(efficientnet, test_loader, device)